# Рекомендация тарифов

## Описание проекта

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.
Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.

## Файл с данными и изучение общей информации

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from joblib import dump
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
df=pd.read_csv('C:\\Users\\ASUS\\OneDrive\\Рабочий стол\\Аналитик Big Data. Яндекс\\Введение в машинное обучение\\users_behavior.csv')
df.head()#первые 5 строк таблицы

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
df.info()#информация о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


### Вывод

Согласно документации к данным:
* сalls — количество звонков,
* minutes — суммарная длительность звонков в минутах,
* messages — количество sms-сообщений,
* mb_used — израсходованный интернет-трафик в Мб,
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Разделение исходных данных

In [4]:
df_train, df_v = train_test_split(df, test_size=0.4, random_state=12345)#разделение 60% на обучающую выборку

In [5]:
df_valid, df_test = train_test_split(df_v, test_size=0.5, random_state=12345)# разделение валидационной и тестовой выборок по 20%

In [6]:
#features - признаки(столбцы в таблице); target - целевой признак
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

## Исследование качество моделей

### Дерево решений

In [14]:
best_score=0
for depth in range(1, 30):
    model = DecisionTreeClassifier(random_state=12345,max_depth=depth)# создание модели дерева решений

    # обучение модели
    model.fit(features_train,target_train)
    predictions_valid = model.predict(features_valid)# предсказания на валидационной выборке
    

    print("max_depth valid =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid))
    if best_score<accuracy_score(target_valid, predictions_valid):
        best_score=accuracy_score(target_valid, predictions_valid)#наилучший показатель
print('Accuracy наилучшей модели =',best_score)

max_depth valid = 1 : 0.7542768273716952
max_depth valid = 2 : 0.7822706065318819
max_depth valid = 3 : 0.7853810264385692
max_depth valid = 4 : 0.7791601866251944
max_depth valid = 5 : 0.7791601866251944
max_depth valid = 6 : 0.7838258164852255
max_depth valid = 7 : 0.7822706065318819
max_depth valid = 8 : 0.7791601866251944
max_depth valid = 9 : 0.7822706065318819
max_depth valid = 10 : 0.7744945567651633
max_depth valid = 11 : 0.7620528771384136
max_depth valid = 12 : 0.7620528771384136
max_depth valid = 13 : 0.7558320373250389
max_depth valid = 14 : 0.7589424572317263
max_depth valid = 15 : 0.7465007776049767
max_depth valid = 16 : 0.7340590979782271
max_depth valid = 17 : 0.7356143079315708
max_depth valid = 18 : 0.7309486780715396
max_depth valid = 19 : 0.7278382581648523
max_depth valid = 20 : 0.7216174183514774
max_depth valid = 21 : 0.7278382581648523
max_depth valid = 22 : 0.7262830482115086
max_depth valid = 23 : 0.7169517884914464
max_depth valid = 24 : 0.713841368584759
ma

Accuracy наилучшей модели = 0.7853810264385692. Нет смысла делать глубину дерева больше 30, т.к. на 9й глубине ухудшается точность предсказания модели.

### Случайный лес

In [15]:
best_model = None
best_result = 0
for est in range(1, 51):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) # обучение модели с заданным количеством деревьев
    model.fit(features_train,target_train) # обучение модели на тренировочной выборке
    result = model.score(features_valid,target_valid) # качество модели на валидационной выборке
    if result > best_result:
        best_model = model# наилучшая модель
        best_result = result#  наилучшее значение метрики accuracy на валидационных данных

print("Accuracy наилучшей модели на валидационной выборке:", best_result, 'луччшее количество деревьев',best_est)

Accuracy наилучшей модели на валидационной выборке: 0.7947122861586314 луччшее количество деревьев 23


### Вывод

Случайный лес выдает точность предсказания на 0.01 больше, чем дерево решений. А значит качество этой модели будет лучше. Поэтому стоит рассмотреть "случайный лес" на тестовой выборке.

## Проверка качества модели на тестовой выборке

In [12]:
best_model = None
best_result = 0
for est in range(1, 51):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) # обучение модели с заданным количеством деревьев
    model.fit(features_train,target_train) # обучение модели на тренировочной выборке
    result = model.score(features_test,target_test) # качество модели на тестовой выборке
    if result > best_result:
        best_model = model# наилучшая модель
        best_result = result#  наилучшее значение метрики accuracy на тестовых данных

print("Accuracy наилучшей модели на валидационной выборке:", best_result)

Accuracy наилучшей модели на валидационной выборке: 0.7962674961119751


### Вывод

Точность предсказания составляет около 0.796.

## Общий вывод

"Случайный лес" является наилучшим выбором модели, т.к. качество в ней более лучше, чем в модели "дерево решений". Правильность ответов на тестовой выборке "случайного леса" составляет около 0.796.